In [2]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"

def save_fig(fig_id, tight_layout=True):
    path = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID, fig_id + ".png")
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format='png', dpi=300)

In [3]:
def plot_image(image):
    plt.imshow(image, cmap="gray", interpolation="nearest")
    plt.axis("off")

def plot_color_image(image):
    plt.imshow(image.astype(np.uint8),interpolation="nearest")
    plt.axis("off")

import tensorflow as tf

In [4]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/")

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


## Setup the datasets, split into train, validation, test

In [5]:
train_size, validation_size, test_size = mnist.train.num_examples, mnist.validation.num_examples, mnist.test.num_examples

X_train, X_validation, X_test = mnist.train.images, mnist.validation.images, mnist.test.images

y_train, y_validation, y_test = mnist.train.labels, mnist.validation.labels, mnist.test.labels

X_train.shape

(55000, 784)

Parameter defination

In [47]:


# def reshape_image (X):
#     return X.reshape ((-1, 28, 28, 1))
# X_train, X_validation, X_test = reshape_image (X_train), reshape_image (X_validation), reshape_image (X_test)
# X_train.shape

In [48]:
# image defination: 
height, width, channel = 28, 28, 1
n_inputs = height * width

# Two layers CNN:
n_CNN_layers = 2

# One layer_DNN: 
n_fc_layers = 1
n_fc = 128
n_output = 10

In [76]:
reset_graph()
with tf.name_scope ('input'):
    X = tf.placeholder (tf.float32, shape = [None, n_inputs], name = 'X')
    X_reshape = tf.reshape (X, shape = [-1, height, width, channel])
    y = tf.placeholder (tf.int32, shape = [None], name = 'Y')

with tf.name_scope ('CNN'):
    conv1 = tf.layers.conv2d (X_reshape, filters = 32, kernel_size = 3, strides = 1, padding = 'SAME', activation = tf.nn.relu, name = 'Conv1' )
    pool1 = tf.nn.max_pool (conv1, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'VALID', name = 'Pool1')
    conv2 = tf.layers.conv2d (pool1, filters = 32, kernel_size = 3, strides = 1, padding = 'SAME', activation = tf.nn.relu, name = 'Conv2' )
    pool2 = tf.nn.max_pool (conv2, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'VALID', name = 'Pool2')

with tf.name_scope ('fc1'):
    pool2_flat = tf.reshape (pool2, shape = [-1, 7 * 7 * 32])
    fc1 = tf.layers.dense (pool2_flat, n_fc, activation = tf.nn.relu, name = 'fc1')

with tf.name_scope ('output'):
    output = tf.layers.dense (fc1, n_output, name = 'output')
    Y_prob = tf.nn.softmax (output, name = 'Y_prob')

with tf.name_scope ('train'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits (logits = output, labels = y)
    loss = tf.reduce_mean (xentropy)
    optimizer = tf.train.GradientDescentOptimizer (learning_rate = 0.01)
    training_op = optimizer.minimize (loss)

with tf.name_scope ('eval'):
    correct = tf.nn.in_top_k (output, y, 1)
    accuracy = tf.reduce_mean (tf.cast (correct, tf.float32))

In [77]:
with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

In [78]:
X_train[0:10].shape

(10, 784)

In [97]:
# Training parameter define
import math
steps = 10
batch_size = 32
n_batch = int (math.ceil (train_size / batch_size ))
print (train_size, n_batch, validation_size)

55000 1719 5000


In [99]:
import time 
with tf.Session() as sess:
    init.run()
    total_start_time = time.time()
    for step in range (steps):
        validation_score = 0 
        start_time = time.time()
        for i_batch in range (n_batch):
            start = i_batch * batch_size
            if i_batch != n_batch - 1:
                end = start + batch_size
            else:
                end = train_size
            sess.run (training_op, feed_dict = {X: X_train[start: end], y: y_train[start: end]})
        train_score = accuracy.eval (feed_dict = {X: X_train[start: end], y: y_train[start: end]})
        for vali_set in range (5):
            validation_score += accuracy.eval (feed_dict = {X: X_validation[vali_set * 1000: (vali_set + 1) * 1000], y:y_validation[vali_set * 1000: (vali_set + 1) * 1000]})
        end_time = time.time ()
        run_time = end_time - start_time
        print ('At step ', step, 'the train_score is ', train_score, 'the validation score is ', validation_score/ 5.0, 'this cost', run_time)
    total_end_time = time.time ()   
    accuracy_test = 0 
    for test_set in range (10):
        accuracy_test += accuracy.eval (feed_dict = {X: X_test[test_set * 1000: (test_set + 1) * 1000], y: y_test[test_set * 1000: (test_set + 1) * 1000]})
    print ('Run', n_steps,  'steps total process time is',  total_end_time - total_start_time)
    print ('the test set accuracy is', accuracy_test / 10.0)

At step  0 the train_score is  1.0 the validation score is  0.9392000079154968 this cost 25.359567165374756
At step  1 the train_score is  1.0 the validation score is  0.9639999985694885 this cost 25.29587745666504
At step  2 the train_score is  1.0 the validation score is  0.9718000054359436 this cost 25.377333641052246
At step  3 the train_score is  1.0 the validation score is  0.9749999880790711 this cost 25.319183588027954
At step  4 the train_score is  1.0 the validation score is  0.9777999877929687 this cost 25.314664363861084
At step  5 the train_score is  1.0 the validation score is  0.9796000123023987 this cost 25.23270010948181
At step  6 the train_score is  1.0 the validation score is  0.9809999942779541 this cost 25.258939504623413
At step  7 the train_score is  1.0 the validation score is  0.9825999855995178 this cost 25.567278385162354
At step  8 the train_score is  1.0 the validation score is  0.9825999855995178 this cost 25.503719329833984
At step  9 the train_score is 

NameError: name 'n_steps' is not defined

In [19]:
X_train.shape

(55000, 784)